In [ ]:
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import named_arrays as na
import astropy.units as u
import astropy.time
import astropy.visualization
import optika
import iris
import esis

In [ ]:
astropy.visualization.quantity_support();

In [ ]:
obs = iris.sg.SpectrographObservation.from_time_range(
    time_start=astropy.time.Time("2021-09-23T14:00"),
    time_stop=astropy.time.Time("2021-09-23T14:01"),
)

obs = obs.explicit

obs = na.despike(obs, axis=(obs.axis_wavelength, obs.axis_detector_y))

where_vertices = np.abs(obs.velocity_doppler.mean(obs.axis_time)) < (100 * u.km / u.s)
where_centers = np.abs(obs.velocity_doppler.cell_centers().mean(obs.axis_time)) < (100 * u.km / u.s)

obs.inputs.wavelength = obs.inputs.wavelength[where_vertices]
obs.outputs = obs.outputs[where_centers]

obs.shape

In [ ]:

# Display the raster sequence as a false-color animation
obs.to_jshtml(
    velocity_min=-100 * u.km / u.s,
    velocity_max=+100 * u.km / u.s,
    fps=2,
)


In [ ]:
wavelength_o_v = 629.73 * u.AA
width_o_v = 150 * u.km / u.s
width_si_iv = 40 * u.km / u.s
velocity = (width_o_v / width_si_iv) * obs.velocity_doppler
obs.inputs.wavelength = velocity.to(u.AA, equivalencies=u.doppler_radio(wavelength_o_v))
obs.wavelength_center = wavelength_o_v

In [ ]:
veranazza_radiance = 690.80 * u.erg / u.cm ** 2 / u.sr / u.s

In [ ]:
dw = np.diff(obs.inputs.wavelength, axis=obs.axis_wavelength)
avg = np.nanmean((dw * obs.outputs).sum(obs.axis_wavelength)).ndarray
avg

In [ ]:
obs = obs / avg * veranazza_radiance
obs = obs.to(u.erg / u.cm ** 2 / u.sr / u.s / u.AA)

In [ ]:
obs.outputs = np.nan_to_num(obs.outputs)

In [ ]:
np.nanmean((dw * obs.outputs).sum(obs.axis_wavelength)).ndarray

In [ ]:
obs.inputs.position.x -= obs.inputs.position.x.mean((obs.axis_detector_x, obs.axis_detector_y))
obs.inputs.position.y -= obs.inputs.position.y.mean((obs.axis_detector_x, obs.axis_detector_y))

In [ ]:
instrument = esis.flights.f1.optics.design_single(num_distribution=0)
instrument.primary_mirror.translation = na.nominal(instrument.primary_mirror.translation)
instrument.field_stop.translation = na.nominal(instrument.field_stop.translation)
instrument.grating.sag.radius = na.nominal(instrument.grating.sag.radius)
instrument.grating.rulings.spacing.coefficients[0] = na.nominal(instrument.grating.rulings.spacing.coefficients[0])
instrument.grating.rulings.spacing.coefficients[1] = na.nominal(instrument.grating.rulings.spacing.coefficients[1])
instrument.grating.rulings.spacing.coefficients[2] = na.nominal(instrument.grating.rulings.spacing.coefficients[2])
instrument.grating.rulings.depth = na.nominal(instrument.grating.rulings.depth)
instrument.grating.rulings.ratio_duty = na.nominal(instrument.grating.rulings.ratio_duty)
instrument.grating.translation = na.nominal(instrument.grating.translation)
instrument.grating.roll = na.nominal(instrument.grating.roll)
instrument

In [ ]:
system = instrument.system
# system.sensor.material = optika.sensors.IdealImagingSensorMaterial()

In [ ]:
image = system.image(
    obs,
    pupil = na.Cartesian2dVectorLinearSpace(
        start=-1,
        stop=1,
        axis=na.Cartesian2dVectorArray("pupil_x", "pupil_y"),
        num=2,
        centers=True,
    ),
    axis_wavelength=obs.axis_wavelength,
    axis_field=(obs.axis_detector_x, obs.axis_detector_y),
)

In [ ]:
r = obs.copy()
factor = 10000
r.outputs = factor * np.nan_to_num(r.outputs)
r.inputs.wavelength = (r.velocity_doppler/factor).to(u.AA, equivalencies=u.doppler_radio(wavelength_o_v))

image_undispersed = system.image(
    r,
    pupil = na.Cartesian2dVectorLinearSpace(
        start=-1,
        stop=1,
        axis=na.Cartesian2dVectorArray("pupil_x", "pupil_y"),
        num=2,
        centers=True,
    ),
    axis_wavelength=obs.axis_wavelength,
    axis_field=(obs.axis_detector_x, obs.axis_detector_y),
    noise=False,
)

In [ ]:
crop = dict(
    detector_x=slice(1024 + 256, 1024 + 512 + 256),
    detector_y=slice(256, 512 + 256),
)
image = image[crop]
image_undispersed = image_undispersed[crop]

In [ ]:
stacked = na.stack(
    arrays=[
        image_undispersed,
        image,
    ],
    axis="dispersion",
)

In [ ]:
fig, ax = na.plt.subplots(
    axis_cols="dispersion",
    ncols=3,
    # sharex=True,
    # sharey=True,
    figsize=(9, 4.3),
    constrained_layout=True,
    dpi=200,
    gridspec_kw=dict(width_ratios=[.9, .9, .1]),
)
cax = ax.ndarray[~0]
cax_twin = cax.twinx()
ax = ax[dict(dispersion=slice(0, 2))]
cax.xaxis.set_ticks_position("top")
cax.xaxis.set_label_position("top")
ani, colorbar = na.plt.rgbmovie(
    obs.inputs.time,
    image.inputs.wavelength,
    image.inputs.position.x.to(u.mm),
    image.inputs.position.y.to(u.mm),
    C=stacked.outputs,
    axis_time=obs.axis_time,
    axis_wavelength=obs.axis_wavelength,
    ax=ax,
    vmin=0,
    vmax=np.nanpercentile(image.outputs, 99.9, axis=(obs.axis_time, "detector_x", "detector_y")),
)
colorbar = colorbar[{obs.axis_time: 0}]
na.plt.pcolormesh(
    colorbar.inputs.x,
    colorbar.inputs.y,
    C=colorbar.outputs.value,
    axis_rgb=obs.axis_wavelength,
    ax=cax,
)
na.plt.pcolormesh(
    colorbar.inputs.x,
    colorbar.inputs.y.to(u.km / u.s, equivalencies=u.doppler_radio(wavelength_o_v)),
    C=colorbar.outputs.value,
    axis_rgb=obs.axis_wavelength,
    ax=cax_twin,
)
na.plt.set_xlim(5.5, 9, ax=ax)
na.plt.set_ylim(-2, 2, ax=ax)
na.plt.set_aspect("equal", ax=ax)
na.plt.set_xlabel("detector $x$ (mm)", ax=ax)
na.plt.set_ylabel("detector $y$ (mm)", ax=ax.ndarray[0])
na.plt.set_ylabel("", ax=ax.ndarray[~0])
ax.ndarray[~0].tick_params(axis="y", labelleft=False)

result = ani.to_jshtml(fps=2)
result = IPython.display.HTML(result)

plt.close(ani._fig)

result

In [ ]:
t0 = {obs.axis_time: 1}
s = stacked[t0]

fig, ax = plt.subplots(
    ncols=2,
    constrained_layout=True,
    dpi=200,
    gridspec_kw=dict(width_ratios=[.9, .1]),
    figsize=(7, 5),
)
ax_twin = ax[1].twinx()
ax[1].xaxis.set_ticks_position("top")
ax[1].xaxis.set_label_position("top")
ani, colorbar = na.plt.rgbmovie(
    obs.inputs.time[t0],
    image.inputs.wavelength[t0],
    s.inputs.position.x.to(u.mm),
    s.inputs.position.y.to(u.mm),
    C=na.nominal(s.outputs),
    axis_time="dispersion",
    axis_wavelength=obs.axis_wavelength,
    ax=ax[0],
    vmin=0,
    vmax=np.nanpercentile(image.outputs, 99.9, axis=(obs.axis_time, "detector_x", "detector_y")),
)
# colorbar = colorbar[{obs.axis_time: 0}]
na.plt.pcolormesh(
    colorbar.inputs.x,
    colorbar.inputs.y,
    C=colorbar.outputs.value,
    axis_rgb=obs.axis_wavelength,
    ax=ax[1],
)
na.plt.pcolormesh(
    colorbar.inputs.x,
    colorbar.inputs.y.to(u.km / u.s, equivalencies=u.doppler_radio(wavelength_o_v)),
    C=colorbar.outputs.value,
    axis_rgb=obs.axis_wavelength,
    ax=ax_twin,
)
na.plt.set_xlim(5.5, 9, ax=ax[0])
na.plt.set_ylim(-2, 2, ax=ax[0])
na.plt.set_aspect("equal", ax=ax[0])
na.plt.set_xlabel(f"detector $x$ (mm)", ax=ax[0])
na.plt.set_ylabel("detector $y$ (mm)", ax=ax[0])

result = ani.to_jshtml(fps=.5)
result = IPython.display.HTML(result)

plt.close(ani._fig)

result